<a href="https://colab.research.google.com/github/redinbluesky/the-lm-book/blob/main/03_순환_신경망.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차
* [Chapter 3_1 엘만 RNN](#chapter3_1)
* [Chapter 3_2 미니 배치 경사 하강법](#chapter3_2)
* [Chapter 3_3 RNN 구현하기](#chapter3_3)
* [Chapter 3_4 RNN 언어모델](#chapter3_4)
* [Chapter 3_5 임베딩 층](#chapter3_5)
* [Chapter 3_6 RNN 언어모델 훈련시키기](#chapter3_6)
* [Chapter 3_7 Dataset과 DataLoader](#chapter3_7)
* [Chapter 3_8 훈련 데이터와 손실 계산](#chapter3_8)

## Chapter 3_1 엘만 RNN <a class="anchor" id="chapter3_1"></a>
1. 순환 신경망(Recurrent Neural Network, RNN)은 순차 데이터(sequential data)를 위해 고안된 신경망이다.
    - 유닛 사이에 있는 연결에 루프가 포함되어 있어 시퀸스에서 한 단계의 정보가 다음 단계로 전달 될 수 있다.
    - 시계열 분석, 자연어 처리와 같은 분야에서 주로 사용된다.

2. 하나의 유닛을 가진 신경망과 입력 문서 "Learning from text is cool"이 있다고 가정하자.
    - 대소문자와 구두점을 무시하고 이 문서를 다음과 같은 행렬로 표현할 수 있다.
    <pre>
         단어     임베딩 벡터
         learning [0.1, 0.2, 0.6]
         from     [0.2, 0.1, 0.4]
         text     [0.1, 0.3, 0.5]
         is       [0.0, 0.7, 0.1]
         cool     [0.5, 0.2, 0.7]
         PAD      [0.0, 0.0, 0.0] (패딩 토큰)
    </pre>
    - 각 행은 신경망 훈련 과정에서 학습된 단어 임베딩 벡터이다.
    - 단어의 순서는 보존되며, 행렬 차원은 (시퀸스 길이, 임베딩차원) =(6, 3)이다.
       - 시퀸스 길이는 문서에 있는 최대 단어 수
    - 시퀸스 길이보다 짧은 문서는 패딩 토큰(PAD)으로 채운다.
       - 패딩 토큰은 더미 임베딩으로 일반적으로 0 벡터를 사용한다.
    - 수학적으로 행렬을 표현하면 아래와 같다.
    <pre>
    X = [ [0.1, 0.2, 0.6],
          [0.2, 0.1, 0.4],
          [0.1, 0.3, 0.5],
          [0.0, 0.7, 0.1],
          [0.5, 0.2, 0.7],
          [0.0, 0.0, 0.0] ]
    </pre>
    - 5개의 3차원 임베딩 벡터 x₁, x₂, x₃, x₄, x₅가 문서에 있는 단어를 나타낸다.
        - learning / x₁ = [0.1, 0.2, 0.6]ᵀ, from / x₂ = [0.2, 0.1, 0.4]ᵀ, text / x₃ = [0.1, 0.3, 0.5]ᵀ, 
        - is / x₄ = [0.0, 0.7, 0.1]ᵀ, cool / x₅ = [0.5, 0.2, 0.7]ᵀ

3. 엘만 RNN(Elman RNN)은 순환 신경망의 한 종류이다.
    - 아래의 그림과 같이 임베딩 벡터의 시퀸스를 한번에 하나씩 처리한다.
    - 각 타임 스텝 t에서 현재 입력 임베딩 벡터 xₜ와 이전 은닉 상태 hₜ₋₁를 훈련 가능한 가중치 Wₕ, Uₕ와 곱해진 후 편향 벡터 b와 더해져 은닉층 hₜ이 계산된다.
    - 스칼라를 출력하는 MLP 유닛과 달리 RNN 유닛은 벡터를 출력하고 하나의 층처럼 동작한다.
    - 초기 은닉상태 h₀는 일반적으로 0 벡터로 설정된다.

        ![Elman RNN](image/03-01-엘만_RNN.png)

    - 은닉 상태는 시퀸스에 있는 이전 단계의 정보를 기억하는 메모리 벡터이다.
    - 현재 입력과 지난 은닉 상태를 사용해 매 스탭마다 업데이트 되며, 신경망이 앞선 단어의 문맥을 사용해 다음 단어를 예측할 수 있도록 돕는다.
    - 심층 신경망을 만들려면 두 번째 RNN 층을 추가한다. 첫 번째 층의 출력이 두 번째 층의 입력이 된다.

        ![Stacked Elman RNN](image/03-01-스택_엘만_RNN.png)

## Chapter 3_2 미니 배치 경사 하강법 <a class="anchor" id="chapter3_2"></a>
1. 미니 배치 경사 하강법(Mini-batch Gradient Descent)은 신경망 훈련에 자주 사용되는 최적화 알고리즘이다.
    - 데이터의 작은 부분집합에서 도함수를 계산하여 학습 속도를 높이고 메모리 사용량을 줄인다.
    - 데이터 크기는 (미니 배치 크기, 시퀸스 길이, 임베딩 차원)의 형태가 된다.
    - 훈련 세트를 고정된 크기의 미니 배치로 나누고, 각 미니 배치에는 일정한 길이의 시퀸스가 포함된다.

2. 예를 들어 배치 크기 2, 시퀸스 길이 4, 임베딩 차원 3이라면 미니 배치는 다음과 같이 나타낼 수 있다.
    <pre>
    Batch = [ [seq1,1, seq1,2, seq1,3, seq1,4],
              [seq2,1, seq2,2, seq2,3, seq2,4] ]
    </pre>
    - seqi,j는 i∈{1,2}이고, j∈{1,2,3,4}인 임베딩 벡터이다.
    - 각 시퀸스의 임베딩 벡터가 다음과 같다고 가정한다.
    <pre>
    seq1 = [[0.1, 0.2, 0.3]
            [0.4, 0.5, 0.6]
            [0.7, 0.8, 0.9]
            [0.0, 1.1, 1.2]]
    seq2 = [[1.3, 1.4, 1.5]
            [1.6, 1.7, 1.8]    
            [1.9, 2.0, 2.1]
            [2.2, 2.3, 2.4]]
    </pre>
    - 이 경우 미니 배치는 다음과 같이 표현할 수 있다.
    <pre>
    Batch = [ [[0.1, 0.2, 0.3],
               [0.4, 0.5, 0.6],
               [0.7, 0.8, 0.9],
               [0.0, 1.1, 1.2]],    
              [[1.3, 1.4, 1.5],
               [1.6, 1.7, 1.8],    
               [1.9, 2.0, 2.1],
               [2.2, 2.3, 2.4]] ]
    </pre>
    - 미니 배치의 차원은 (2, 4, 3)이 된다.

3. 경사 하강법 단계마다 다음을 수행한다.
    - 훈련 세트에서 미니 배치를 추출한다.
    - 이를 신경망에 통과시킨다.
    - 손실을 계산한다.
    - 그레이디언트를 계산한다.
    - 모델 파라미터를 업데이트한다.
    - 이 과정을 훈련 세트 전체에 대해 반복한다.

4. 매 단계마다 전체 훈련 세트를 사용하는 경우보다 미니 배치 경사 하강법이 더 빠를게 수렴되는 경우가 많다.
    - 병렬처리 기능을 활용하여 대규모 모델과 데이터셋을 효율적으로 다룰 수 있다.


## Chapter 3_3 RNN 구현하기 <a class="anchor" id="chapter3_3"></a>

In [1]:
import torch
import torch.nn as nn

class ElmanRNNUnit(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.Uh = nn.Parameter(torch.randn(emb_dim, emb_dim)) # 은닉 상태로 가는 가중치
        self.Wh = nn.Parameter(torch.randn(emb_dim, emb_dim)) # 입력으로 가는 가중치
        self.b = nn.Parameter(torch.zeros(emb_dim)) # 편향
        
    def forward(self, x, h):
        # 엘만 RNN의 순전파 계산
        #   - 각각 크기가 (배치 크기, 임베딩 차원)인 입력 x와 이전 은닉 상태 h에 가중치 행렬을 곱하고 
        #     편향을 더한 후, 하이퍼볼릭 탄젠트 활성화 함수를 적용
        #   - 출력은 크기가 (배치 크기, 임베딩 차원)인 새로운 은닉 상태
        # @ 연산자는 행렬 곱셈을 나타냄
        #   - x @ self.Wh 배치 입력 x는 크기가 (배치 크기, 임베딩 차원)이고,
        #     self.Wh는 크기가 (임베딩 차원, 임베딩 차원)이므로 결과는 (배치 크기, 임베딩 차원)
        #
        return torch.tanh(x @ self.Wh + h @ self.Uh + self.b)

In [2]:
import torch
import torch.nn as nn

class ElmanRNN_V2(nn.Module):
    """두개의 RNN 층을 가진 엘만 RNN의 구현"""
    def __init__(self, emb_dimm, num_layers):
        super().__init__()
        self.emb_dim = emb_dimm
        self.num_layers = num_layers
        # RNN 층을 num_layers 개수만큼 생성
        self.rnn_units = nn.ModuleList([ElmanRNNUnit(emb_dimm) for _ in range(num_layers)])
        
    def forward(self, x, h):
       batch_size, seq_len, emb_dim = x.shape # 입력 x의 크기에서 배치 크기, 시퀀스 길이, 임베딩 차원 추출
       # 이전 은닉 상태를 모두 0으로 초기화
       #   - 각 층의 은닉상태는 훈련 과정에서 업데이트 해야 하므로 다차원 텐서 대신 리스트에 저장한다.
       #   - 텐서 값을 수정하면 파이토치의 자동 미분 시스템에 문제를 발생시켜 그래디언트 계산에 영향을 줄 수 있다.
       h_prev = [
           torch.zeros(batch_size, emb_dim, device=x.device)
           for _ in range(self.num_layers)
       ]
       outputs = []
       # 시퀀스 길이만큼 반복
       for t in range(seq_len):
           input_t = x[:, t] # 현재 시점의 입력 추출
           # 각 RNN 층을 순차적으로 통과
           for l, run_unit in enumerate(self.rnn_units):
               h_new = run_unit(input_t, h_prev[l]) # 현재 층의 RNN 유닛에 입력과 이전 은닉 상태 전달
               h_prev[l] = h_new # 새로운 은닉 상태로 업데이트
               input_t = h_new  # 다음 층의 입력으로 사용
           outputs.append(input_t)
       return torch.stack(outputs, dim=1)

## Chapter 3_4 RNN 언어모델 <a class="anchor" id="chapter3_4"></a>

In [3]:
class RecurrentLanguageModel(nn.Module):
    """RNN 언어모델의 구현"""
    def __init__(self, vocab_size, emb_dim, num_layers, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx) # 단어 임베딩 층
        self.rnn = ElmanRNN_V2(emb_dim, num_layers) # 엘만 RNN 층
        self.fc = nn.Linear(emb_dim, vocab_size) # 출력층
        
    def forward(self, x, h):
        # 입력 x를 단어 임베딩 층에 통과
        x_emb = self.embedding(x) # 크기: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # RNN 층에 통과
        rnn_out = self.rnn(x_emb, h) # 크기: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # 출력층에 통과
        logits = self.fc(rnn_out) # 크기: (배치 크기, 시퀀스 길이, 어휘 수)
        return logits

## Chapter 3_5 임베딩 층 <a class="anchor" id="chapter3_5"></a>
1. 임베딩 층은 파이토치에서 nn.Embedding 클래스로 구현된다.
    - 어휘사전에서 얻은 토큰 인덱스를 고정된 크기의 밀집 벡터로 매핑한다.
    - 각 토큰에 고유한 임베딩 벡터가 할당되는 학습 가능한 룩업 테이블 처럼 동작한다.
    - 훈련 과정에서 벡터는 업데이트 되어 단어의 의미적 관계를 캡처한다.

2. nn.Embedding 클래스의 주요 매개변수는 다음과 같다.
    - nn.Embedding(num_embeddings, embedding_dim)
        - num_embeddings: 단어 집합의 크기(고유 토큰 수)
        - embedding_dim: 각 단어를 나타내는 임베딩 벡터의 차원

3. 예를 들어 5개의 토큰(인덱스 0~4)과 임베딩 차원 3을 가진 임베딩 층을 생성하려면 다음과 같이 한다.
    - 임베딩 층은 임베딩 행렬을 랜덤한 값으로 초기화 한다.
    - 임베딩 층의 각 행은 고유한 토큰에 해당하는 임베딩 벡터이다.

In [4]:
import torch
import torch.nn as nn

vocab_size = 5 # 고유한 토큰 개수
emb_dim = 3   # 임베딩 벡터의 크기
emb_layer = nn.Embedding(vocab_size, emb_dim) # 임베딩 층 생성
print(emb_layer.weight) # 임베딩 층의 가중치 출력

Parameter containing:
tensor([[ 0.3111, -0.6561,  0.8791],
        [-0.1496,  1.3648, -0.8125],
        [-0.6747,  0.1977,  0.5416],
        [ 0.7518, -0.2731, -0.8682],
        [ 0.3165, -0.8922, -0.5182]], requires_grad=True)


In [5]:
token_indices = torch.tensor([0, 2, 4]) # 예시 토큰 인덱스
embeddings = emb_layer(token_indices) # 토큰 인덱스를 임베딩 벡터로 변환
print(embeddings) # 변환된 임베딩 벡터 출력

tensor([[ 0.3111, -0.6561,  0.8791],
        [-0.6747,  0.1977,  0.5416],
        [ 0.3165, -0.8922, -0.5182]], grad_fn=<EmbeddingBackward0>)


4. 임베딩 층은 패딩 토큰도 다룰 수 있다.
    - 패딩은 미니 배치에 있는 시퀸스가 동일한 길이가 되게 만다.
    - 훈련 과정에서 모델이 패딩 토큰에 대한 임베딩을 업데이트 하지 않도록 패딩 토큰을 영벡터에 매핑한다.

In [6]:
emb_layer = nn.Embedding(vocab_size, emb_dim, padding_idx=0) # 임베딩 행렬의 첫 번째 행이 영벡터가 되도록 설정
token_indices = torch.tensor([0, 2, 4]) # 예시 토큰 인덱스
embeddings = emb_layer(token_indices) # 토큰 인덱스를 임베딩 벡터로 변환
print(embeddings) # 변환된 임베딩 벡터 출력

tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.4305,  0.2512,  0.6894],
        [-0.3420, -0.4225, -0.7762]], grad_fn=<EmbeddingBackward0>)


## Chapter 3_6 RNN 언어모델 훈련시키기 <a class="anchor" id="chapter3_6"></a>

In [ ]:
import torch
import random

def set_seed(seed):
    """재현 가능한 결과를 위해 랜덤 시드 설정"""
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True # CUDA에서 재현 가능하도록 설정
        torch.backends.cudnn.benchmark = False # 성능 최적화 비활성화

In [8]:
# transformers 패키지 설치
#   - 허깅 페이스 허브에서 사전 훈련된 모델을 쉽게 다운로드, 훈련, 사용할 수 있는 API와 도구를 제공하는 오픈 소스 라이브러리
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
def get_hyperparameters():
    """
    모델 훈련을 위한 기본 하이퍼파라미터를 반환합니다.
    Returns:
        tuple: (emb_dim, num_layers, batch_size, learning_rate, num_epochs)
    """
    emb_dim = 128         # 임베딩 차원
    num_layers = 2        # RNN 층 수
    batch_size = 128      # 훈련 배치 크기
    learning_rate = 0.001 # 최적화를 위한 학습률
    num_epochs = 1        # 훈련 에포크 수
    context_size = 30     # 최대 입력 시퀀스 길이
    return emb_dim, num_layers, batch_size, learning_rate, num_epochs, context_size

# ----------------------------
# 순환 언어 모델 클래스
# ----------------------------
def initialize_weights(model):
    """
    다차원 파라미터에는 Xavier 균등 초기화를 사용하고, 편향 및 기타 1차원 파라미터에는
    균등 초기화를 사용하여 모델 가중치를 초기화합니다.
    Args:
        model (nn.Module): 가중치를 초기화해야 하는 파이토치 모델
    """
    # 모델에서 이름을 가진 파라미터를 모두 반복
    for name, param in model.named_parameters():
        # 파라미터가 1차원 이상인 경우 확인 (예: 가중치 행렬)
        if param.dim() > 1:
            # 가중치 행렬에 Xavier 균등 초기화 사용
            # 이는 분산을 일정하게 유지하여 기울기 소실/폭주을 방지하는 데 도움이 됩니다
            nn.init.xavier_uniform_(param)
        else:
            # 1차원 파라미터(편향 등)의 경우 간단한 균등 초기화 사용
            nn.init.uniform_(param)



In [ ]:
from transformers import AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 허깅 페이스 허브에 있는 대부분의 모델은 훈력할 때 사용했던 토크나이저를 포함하고있다.
#   - Phi-3.5-mini-instruct : 바이트페어 인코딩 알고리즘으로 훈련되었으며, 어휘사전의 크기는 32,011이다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct") 
vocat_size = len(tokenizer) # 토크나이저의 어휘 사전 크기
print(f"Vocabulary size: {vocat_size}")

emb_dim, num_layers, batcch_size, learning_rate, num_epochs = get_hyperparameters()

data_url = "https://www.thelmbook.com/data/news"
train_loader, test_loader = download_and_prepare_data(data_url, tokenizer, batch_size)

model = RecurrentLanguageModel(vocab_size, emb_dim, num_layers, pad_idx=tokenizer.pad_token_id)
initialize_weights(model) # 모델 가중치 초기화
model.to(device) # 모델을 GPU 또는 CPU로 이동

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id) # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 옵티마이저 정의

Using device: cuda
Vocabulary size: 32011


## Chapter 3_7 Dataset과 DataLoader <a class="anchor" id="chapter3_7"></a>
1. Dataset 클래스는 실제 데이터 소스에 대한 인터페이스 역활을 한다.
    - __len__ 메서드는 데이터셋의 샘플 수를 반환
    - __getitem__ 메서드는 주어진 인덱스에 해당하는 샘플을 반환
    - 사용자 정의 데이터셋을 만들려면 torch.utils.data.Dataset 클래스를 상속하고 이 두 메서드를 구현해야 한다.

In [15]:
import json
import torch
from torch.utils.data import Dataset

class JSONDataset(Dataset):
    """JSON 파일에서 데이터를 로드하는 사용자 정의 데이터셋 클래스"""
    def __init__(self, file_path):
        self.data = []
        with open(file_path, 'r') as f:
            for line in f:
                item = json.loads(line) # JSON 형식의 각 줄을 파싱
                features = [item['feature1'], item['feature2']] # 특징 추출
                label = item['label'] # 레이블 추출
                self.data.append((features, label)) # 특징과 레이블을 튜플로 저장
            
    def __len__(self):
        return len(self.data) # 데이터셋의 샘플 수 반환
    
    def __getitem__(self, idx):
        features, label = self.data[idx] # 인덱스에 해당하는 샘플 반환
        features = torch.tensor(features, dtype=torch.float32) # 특징 벡터 생성
        label = torch.tensor(label, dtype=torch.float32) # 레이블 생성
        return features, label

In [19]:
dataset = JSONDataset('data.jsonl') # JSON 파일에서 데이터셋 생성
print(f"Dataset size: {len(dataset)} samples") # 데이터셋 크기 출력
features, label = dataset[0] # 첫 번째 샘플 가져오기
print(f"Features: {features}, Label: {label}") # 특징과 레이블 출력

Dataset size: 3 samples
Features: tensor([1., 2.]), Label: 3.0


2. DataLoader는 Dataset과 함께 사용되어 배치, 셔플링, 병렬 로딩과 같은 작업을 처리한다.
    - DataLoader는 훈련 루프에서 반복할 수 있는 이터레이터를 제공한다.
    - 배치 크기, 셔플 여부, 병렬 데이터 로딩을 위한 워커 수 등을 지정할 수 있다.
    - 훈련 프로세스를 간소화므로 모델의 설계와 최적화에만 집중할 수 있다.

In [20]:
from torch.utils.data import DataLoader

dataset = JSONDataset('data.jsonl') # JSON 파일에서 데이터셋 생성
data_loader = DataLoader(dataset, 
                         batch_size=2, # 배치 크기 설정 
                         shuffle=True, # 셔플 여부 설정 
                         num_workers=0 # 병렬 데이터 로딩을 위한 워커 수 설정
                         ) 

num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, (features, labels) in enumerate(data_loader):
        print(f"Batch {batch_idx+1}:")
        print(f"  Features: {features}")
        print(f"  Labels: {labels}")

Epoch 1/3
Batch 1:
  Features: tensor([[4., 5.],
        [1., 2.]])
  Labels: tensor([9., 3.])
Batch 2:
  Features: tensor([[7., 8.]])
  Labels: tensor([15.])
Epoch 2/3
Batch 1:
  Features: tensor([[4., 5.],
        [1., 2.]])
  Labels: tensor([9., 3.])
Batch 2:
  Features: tensor([[7., 8.]])
  Labels: tensor([15.])
Epoch 3/3
Batch 1:
  Features: tensor([[7., 8.],
        [1., 2.]])
  Labels: tensor([15.,  3.])
Batch 2:
  Features: tensor([[4., 5.]])
  Labels: tensor([9.])


## Chapter 3_8 훈련 데이터와 손실 계산 <a class="anchor" id="chapter3_8"></a>
1. 신경망 언어 모델을 공부할 때 훈련 샘플의 구조를 이해하는 것이 중요하다.
    - 텍스트 말뭉치는 중첩된 입력과 타깃 시퀸스로 분할된다.
    - 각 입력 시퀸스에 대해 타깃 시퀸스는 토큰이 하나씩 밀려서 구성된다.
    - 모델이 시퀸스에 있는 각 위치에서 다음 단어를 예측하도록 훈련 할 수 있다.

2. 예를 들어, "We train a recurrent neural network as a language model"이라는 문장이 있다고 가정하자.
    - Phi 3.5 미니 모델로 토큰화 하면 결과는 다음과 같다.
    <pre>
        ['_We', '_train', '_a', '_rec','urrent', '_neural', '_network', '_as', '_a', '_language', '_model', '.']
    </pre>   
    - 입력 시퀸스와 타깃 시퀸스는 다음과 같이 구성된다.
    <pre>
        Input Sequence:  ['_We', '_train', '_a', '_rec','urrent', '_neural', '_network', '_as', '_a', '_language', '_model']
        Target Sequence: ['_train', '_a', '_rec','urrent', '_neural', '_network', '_as', '_a', '_language', '_model', '.']      
    </pre>

3. 훈련 샘플이 완전한 문장일 필요는 없다.
    - 현대 언어 모델은 한번에 처리할 수 있는 최대 토큰 수(예를들어, 8192)인 문맥 윈도길이 까지 시퀸스를 처리한다.
    - 윈도는 모델이 텍스트에 얼마나 멀리 관계를 맺을 수 있는지를 제한한다.
    - 훈련하는 동안 텍스트를 윈도 크기 청크로 나누며, 각 타깃 시퀸스는 입력에서 토큰을 하나씩 밀어서 생성된다.

4. 훈련 과정에서 RNN은 한 번에 하나의 토큰을 처리하며 은닉 상태를 업데이트 한다.
    - 각 타임 스텝에서 시퀸스에 있는 다음 토큰을 예측하기 위한 로짓을 생성한다.
    - 각 로짓은 어휘사전의 토큰에 대은되며, 소프트맥스를 사용하여 확률로 변환된다.
    - 그 다음 확률을 사용하여 손실을 계산한다.

5. 모델이 "_We"를 처리하고 어휘사전에 있는 모든 토큰에 대해 활률을 할당하여 "_train"을 예측했다고 가정하자.
    - "_train"의 확률을 사용해 손실을 계산한다.
    - 그런다음 모델이 "_train"을 처리하고 다음 토큰 "_a"를 예측하는 과정을 반복한다.
    - 이런 식으로 입력 시퀸스에 있는 모든 토큰에 대해 예측과 손실 계산이 계속된다.

6. 위의 예제의 경우 11개의 예측을 만들고 각 예측에 대해 손실을 계산한다.
    - 이 손실이 훈련 샘플에 있는 토큰과 배체에 있는 모든 샘플에 대해 편균된다.
    - 역전파에서 이 평균 손실을 사용하해 모델 파라미터를 업데이트 한다.

7. 가상의 숫자로 각 위치의 손실 계산을 수행한다.
    <pre>
        Position:        1      2      3      4      5      6      7      8      9     10     11
        Input Token:   '_We' '_train' '_a' '_rec' 'urrent' '_neural' '_network' '_as' '_a' '_language' '_model'
        Target Token:  '_train' '_a' '_rec' 'urrent' '_neural' '_network' '_as' '_a' '_language' '_model' '.'
        Logic          -0.5     3.2   -0.3   -0.02   -0.6   -1.0   -0.4   -0.9   -0.7   -1.3   -0.2
        손실에 대한      2.99    3.91    0.7    0.5    0.4    0.3    0.8    0.6    0.5    0.2    0.9
        기여(-log 확률):   
        Loss:           0.5    0.3    0.2    0.4    0.6    0.3    0.2    0.5    0.4    0.3    0.7
    </pre>
    - 총 손실은 각 위치의 손실을 더한 값인 4.4가 된다.
    - 평균 손실은 총 손실을 위치 수(11)로 나눈 값인 약 0.4가 된다.
    - 이 평균 손실이 역전파에서 모델 파라미터를 업데이트 하는데 사용된다.

8. 훈련 과정의 목표는 손실을 최소화하는 것이다.
    - 각 위치에서 올바른 타깃 토큰에 높은 확률을 할당하도록 모델을 훈련시킨다.

10. 엘만 RNN의 단점
    - 모델의 파리미터 수는 8,292,619개로 작으며, 대 부분이 임베딩 층의 파라미터이다.
    - 문맥 윈도 크기가 비교적 작다.(30 토큰)
    - 엘만 RNN의 은직 상태는 초기 토큰에서 얻은 정보를 점진적으로 잊어버린다.

11. minLSTM과 xLSTM 구조가 2024년에 개발되면서 RNN에 대한 관심이 늘어나고있다.